In [2]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3


In [81]:

body = open('news.txt').read().split('\n')

body[:5]

['u\'(CNN) The synthetic cannabinoids often called "fake weed" are a mix of chemicals sprinkled on what looks like incense and sold in shiny packages, often to teens and young adults.\\n\\nIt produces sensations similar to tetrahydrocannabinol (THC), the psychoactive ingredient in the cannabis plant, yet many users report more powerful, often dangerous effects.\\n\\nNew research from the Centers for Disease Control and Prevention says young people who use fake weed are also more likely to use other drugs or alcohol, to behave violently and to have high-risk sex, according to a study published Monday in the journal Pediatrics\\n\\nTo understand the overall behavior of fake weed users, Heather Clayton, lead researcher of the new study and a health scientist in the CDC\\\'s Division of Adolescent and School Health, turned to the 2015 Youth Risk Behavior Survey, a school-based anonymous questionnaire administered to a nationally representative sample of high school students.\\n\\nA total o

In [82]:
comments = [x.decode('UTF-8','ignore') for x in body]
#title = [x.decode('UTF-8','ignore') for x in ids]

In [144]:
str(len(ids))

NameError: name 'ids' is not defined

In [83]:
str(len(comments))

'30'

In [84]:
# generates index for each item in the corpora (in this case it's just rank) and I'll use this for scoring later
ranks = []

for i in range(0,len(comments)):
    ranks.append(i)

In [85]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(['diabetes','use','said','say','says'])


In [86]:
stemmer = nltk.PorterStemmer()

In [87]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    words = [w.lower() for w in tokens if w.isalpha() if w.lower()not in stopwords]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in words:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    words = [w.lower() for w in tokens if w.isalpha() if w.lower()not in stopwords]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in words:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [88]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in comments:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [89]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)


# Tf-idf and document similarity¶

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.99, max_features=200000,
                                 min_df=0.3, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(comments)


Wall time: 628 ms


In [91]:
terms = tfidf_vectorizer.get_feature_names()

In [92]:
terms

[u'accord',
 u'ad',
 u'adult',
 u'affect',
 u'associ',
 u'becom',
 u'blood',
 u'blood glucos',
 u'blood glucos level',
 u'blood sugar',
 u'bodi',
 u'care',
 u'caus',
 u'chang',
 u'condit',
 u'consid',
 u'continu',
 u'control',
 u'damag',
 u'data',
 u'day',
 u'develop',
 u'diabet',
 u'diet',
 u'diseas',
 u'doctor',
 u'drink',
 u'drug',
 u'eat',
 u'effect',
 u'especi',
 u'factor',
 u'food',
 u'gener',
 u'glucos',
 u'glucos level',
 u'health',
 u'healthi',
 u'heart',
 u'help',
 u'high',
 u'hospit',
 u'import',
 u'includ',
 u'increas',
 u'inform',
 u'insulin',
 u'know',
 u'lead',
 u'level',
 u'like',
 u'limit',
 u'live',
 u'look',
 u'make',
 u'manag',
 u'mani',
 u'medic',
 u'medicin',
 u'million',
 u'need',
 u'new',
 u'number',
 u'obes',
 u'patient',
 u'peopl',
 u'plan',
 u'prevent',
 u'produc',
 u'research',
 u'result',
 u'risk',
 u'sever',
 u'start',
 u'studi',
 u'sugar',
 u'sure',
 u'test',
 u'time',
 u'treat',
 u'treatment',
 u'tri',
 u'type',
 u'univers',
 u'use',
 u'weight',
 u'work'

In [93]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

# K-means clustering

In [94]:
from sklearn.cluster import KMeans

num_clusters =5

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

Wall time: 58 ms


In [95]:
from sklearn.externals import joblib

joblib.dump(km,  'doc_cluster.pkl')
km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [96]:

VA = { 'Rank':ranks, 'Content':comments, 'cluster': clusters }

frame = pd.DataFrame(VA, index = [clusters] , columns = ['Rank','cluster'])
frame.head()

,Rank,cluster
3,0,3
3,1,3
2,2,2
3,3,3
1,4,1


In [97]:
frame['cluster'].value_counts()

2    12
3     7
0     5
1     4
4     2
Name: cluster, dtype: int64

In [71]:
grouped = frame['Rank'].groupby(frame['cluster'])

grouped.mean()

cluster
0    14.25
1     6.00
2     7.40
3     7.25
Name: Rank, dtype: float64

In [98]:
from __future__ import print_function

print("Top terms per cluster:")
print()
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    for ind in order_centroids[i, :6]:
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print()
    print()
    #print("Cluster %d titles:" % i, end='')
    #for title in frame.ix[i]['Title'].values.tolist():
        #print(' %s,' % title, end='')
    #print()
    #print()

Top terms per cluster:

Cluster 0 words: hospital, people, patients, treating, types, care,

Cluster 1 words: medical, drugs, patients, doctoral, study, blood,

Cluster 2 words: glucose, blood, insulin, blood, people, level,

Cluster 3 words: study, used, research, risk, people, factors,

Cluster 4 words: million, number, disease, new, prevention, people,



In [99]:
import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn.manifold import MDS

MDS()

# two components as we're plotting points in a two-dimensional plane
# "precomputed" because we provide a distance matrix
# we will also specify `random_state` so the plot is reproducible.
mds = MDS(n_components=2, dissimilarity="precomputed",n_jobs=-2)

pos = mds.fit_transform(dist)  # shape (n_components, n_samples)

xs, ys = pos[:, 0], pos[:, 1]

In [100]:
#set up colors per clusters using a dict
cluster_colors = {0: '#1b9e77', 1: '#d95f02', 2: '#7570b3', 3: '#e7298a', 4: '#66a61e'}

#set up cluster names using a dict
cluster_names = {0: 'hospital, people, patients, treating, types, care', 
                 1: 'medical, drugs, patients, doctoral, study, blood',
                 2: 'glucose, blood, insulin, blood, people, level',
                 3: 'study, used, research, risk, people, factors',
                 4: 'million, number, disease, new, prevention, people'
                 }
                

In [101]:
#create data frame that has the result of the MDS plus the cluster numbers and titles
df = pd.DataFrame(dict(x=xs, y=ys, label=clusters, title=ranks)) 

#group by cluster
groups = df.groupby('label')

In [102]:
df.head()

,label,title,x,y
0,3,0,0.499284,-0.169480
1,3,1,0.458298,0.371083
2,2,2,-0.131519,-0.590190
3,3,3,0.311212,-0.607585
4,1,4,-0.361328,0.406773


In [103]:
# set up plot
fig, ax = plt.subplots(figsize=(17, 9)) # set size
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling

#iterate through groups to layer the plot
#note that I use the cluster_name and cluster_color dicts with the 'name' lookup to return the appropriate color/label
for name, group in groups:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=cluster_names.get(name), color=cluster_colors.get(name), mec='none')
    ax.set_aspect('auto')
    ax.tick_params(\
        axis= 'x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelbottom='off')
    ax.tick_params(\
        axis= 'y',         # changes apply to the y-axis
        which='both',      # both major and minor ticks are affected
        left='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelleft='off')
    
ax.legend(numpoints=1)  #show legend with only 1 point

#add label in x,y position with the label as the film title
for i in range(len(df)):
    ax.text(df.ix[i]['x'], df.ix[i]['y'], df.ix[i]['title'], size=8)  

    
    
#plt.show() #show the plot

#uncomment the below to save the plot if need be
#plt.savefig('clusters_small_noaxes.png', dpi=200)

In [104]:
#define custom toolbar location
class TopToolbar(mpld3.plugins.PluginBase):
    """Plugin for moving toolbar to top of figure"""

    JAVASCRIPT = """
    mpld3.register_plugin("toptoolbar", TopToolbar);
    TopToolbar.prototype = Object.create(mpld3.Plugin.prototype);
    TopToolbar.prototype.constructor = TopToolbar;
    function TopToolbar(fig, props){
        mpld3.Plugin.call(this, fig, props);
    };

    TopToolbar.prototype.draw = function(){
      // the toolbar svg doesn't exist
      // yet, so first draw it
      this.fig.toolbar.draw();

      // then change the y position to be
      // at the top of the figure
      this.fig.toolbar.toolbar.attr("x", 150);
      this.fig.toolbar.toolbar.attr("y", 400);

      // then remove the draw function,
      // so that it is not called again
      this.fig.toolbar.draw = function() {}
    }
    """
    def __init__(self):
        self.dict_ = {"type": "toptoolbar"}

In [105]:
#create data frame that has the result of the MDS plus the cluster numbers and titles
df = pd.DataFrame(dict(x=xs, y=ys, label=clusters, title=ranks)) 

#group by cluster
groups = df.groupby('label')

#define custom css to format the font and to remove the axis labeling
css = """
text.mpld3-text, div.mpld3-tooltip {
  font-family:Arial, Helvetica, sans-serif;
}

g.mpld3-xaxis, g.mpld3-yaxis {
display: none; }

svg.mpld3-figure {
margin-left: -200px;}
"""

# Plot 
fig, ax = plt.subplots(figsize=(14,6)) #set plot size
ax.margins(0.03) # Optional, just adds 5% padding to the autoscaling

#iterate through groups to layer the plot
#note that I use the cluster_name and cluster_color dicts with the 'name' lookup to return the appropriate color/label
for name, group in groups:
    points = ax.plot(group.x, group.y, marker='o', linestyle='', ms=18,label=cluster_names.get(name), mec='none',color=cluster_colors.get(name))
    ax.set_aspect('auto')
    Labels = [i for i in group.title]
    
    
    #set tooltip using points, labels and the already defined 'css'
    tooltip = mpld3.plugins.PointHTMLTooltip(points[0], Labels,
                                       voffset=10, hoffset=10, css=css)
    #connect tooltip to fig
    mpld3.plugins.connect(fig, tooltip, TopToolbar())    
    
    #set tick marks as blank
    ax.axes.get_xaxis().set_ticks([])
    ax.axes.get_yaxis().set_ticks([])
    
    #set axis as blank
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)

    
ax.legend(numpoints=1) #show legend with only one dot

mpld3.display() #show the plot

#uncomment the below to export to html
#html = mpld3.fig_to_html(fig)
#print(html)